In [12]:
import polars as pl

In [13]:
pitching = pl.read_csv("data/pitching.csv")
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [14]:
eras = (pitching
        .select(['yearID', 'R'])
        .filter((pl.col('yearID') >= 1900) & (pl.col('yearID') < 1939))
        .with_columns(era = pl.when(pl.col('yearID') < 1920)
                              .then(pl.lit('dead ball'))
                              .otherwise(pl.lit('live ball'))
                     )
       )
eras.head()

yearID,R,era
i64,i64,str
1905,22,"""dead ball"""
1909,14,"""dead ball"""
1911,15,"""dead ball"""
1923,5,"""live ball"""
1920,40,"""live ball"""


In [15]:
mean_runs = (eras
             .group_by(pl.col('era'))
             .agg(mean_runs = pl.col('R').mean())
            )
mean_runs

era,mean_runs
str,f64
"""live ball""",55.76718
"""dead ball""",49.707696


In [16]:
pitching.with_columns(
    dh_in_league=(
        pl.when((pl.col("lgID") == "AL") & (pl.col("yearID") >= 1973)).then(pl.lit("yes"))
          .when(pl.col("yearID") == 2020).then(pl.lit("yes"))
          .when((pl.col("lgID") == "NL") & (pl.col("yearID") >= 2022)).then(pl.lit("yes"))
          .otherwise(pl.lit("no"))
          .alias("dh_in_league")
    )
).head()


playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP,dh_in_league
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1,"""no"""
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2,"""no"""
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1,"""yes"""
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4,"""yes"""
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2,"""yes"""


In [17]:
ERA = (pitching
 .filter(pl.col("yearID") >= 1946)
 .group_by(pl.col("yearID", "lgID"))
 .agg
    (total_runs = pl.col("R").sum(),
     total_outs = pl.col("IPouts").sum()
    )
 .with_columns(ERA = (pl.col("total_runs") / pl.col("total_outs")) * 27)
 .with_columns(
    dh_in_league=(
        pl.when((pl.col("lgID") == "AL") & (pl.col("yearID") >= 1973)).then(pl.lit("yes"))
          .when(pl.col("yearID") == 2020).then(pl.lit("yes"))
          .when((pl.col("lgID") == "NL") & (pl.col("yearID") >= 2022)).then(pl.lit("yes"))
          .otherwise(pl.lit("no"))
          .alias("dh_in_league")
    )
)
    .select(["yearID", "lgID", "dh_in_league", "ERA"])
)
ERA

yearID,lgID,dh_in_league,ERA
i64,str,str,f64
1967,"""AL""","""no""",3.693277
1981,"""NL""","""no""",3.894603
2011,"""NL""","""no""",4.170883
2008,"""AL""","""yes""",4.721562
2003,"""AL""","""yes""",4.922077
…,…,…,…
1947,"""NL""","""no""",4.652088
1952,"""NL""","""no""",4.209594
1951,"""NL""","""no""",4.479828


In [18]:
ERA.write_csv("data/DH.csv")

In [19]:
batting = pl.read_csv("data/Batting.csv")
batting

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zwilldu01""",1915,1,"""CHF""","""FL""",150,null,548,65,157,32,7,13,94,24,null,67,65,null,2,18,null,null,null
"""zwilldu01""",1916,1,"""CHN""","""NL""",35,null,53,4,6,1,0,1,8,0,null,4,6,null,0,2,null,null,null
"""zychto01""",2015,1,"""SEA""","""AL""",13,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [21]:
slugging = (
    batting
    .with_columns(
        (
            pl.col("H") - (
                pl.col("2B") +
                pl.col("3B") +
                pl.col("HR")
            )
        ).alias("1B")
    )
    # now filter the dataframe to remove rows where H is 0 or null
    .filter(pl.col("H") > 0)
)

slugging.select(["playerID", "yearID", "H", "1B", "2B", "3B", "HR"])



playerID,yearID,H,1B,2B,3B,HR
str,i64,i64,i64,i64,i64,i64
"""aaronha01""",1954,131,85,27,6,13
"""aaronha01""",1955,189,116,37,9,27
"""aaronha01""",1956,200,126,34,14,26
"""aaronha01""",1957,198,121,27,6,44
"""aaronha01""",1958,196,128,34,4,30
…,…,…,…,…,…,…
"""zuverge01""",1958,2,1,0,1,0
"""zwilldu01""",1910,16,11,5,0,0
"""zwilldu01""",1914,185,123,38,8,16
